<a href="https://colab.research.google.com/github/linztjavier-max/BASC0005-London-Air-Inequality/blob/main/2022_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
url = "https://data.london.gov.uk/download/v8pow/87e880c2-34bd-4d86-8895-e8c5344f358e/traffic-flow-borough.xlsx"

cars = pd.read_excel(url, sheet_name="Traffic Flows - Cars")
allv = pd.read_excel(url, sheet_name="Traffic Flows - All vehicles")

def clean_year(col):
    try:
        return int(col.split()[0])
    except:
        return col

cars.columns = [clean_year(c) for c in cars.columns]
allv.columns = [clean_year(c) for c in allv.columns]

years = list(range(2019, 2023))

cars_df = cars[["LA Code", "Local Authority"] + years]
vehicles_df = allv[["LA Code", "Local Authority"] + years]

In [ ]:
#Traffic Flow Car Output
cars_df

In [ ]:
#Traffic Flow All Vehicle Output
vehicles_df

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# data in population
df_path_pop = (
    "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/"
    "populationandmigration/populationestimates/datasets/"
    "populationestimatesforukenglandandwalesscotlandandnorthernireland/"
    "mid2024/mye24tablesuk.xlsx"
)

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(df_path_pop, headers=headers)
response.raise_for_status()

pop_df = pd.read_excel(BytesIO(response.content), sheet_name="MYE5", header=7)


In [ ]:
#Output pandas in population
pop_filtered_df = pop_df.iloc[201:234]
def clean_pop_df_column_name(col_name):
    if isinstance(col_name, str) and col_name.startswith('Mid-'):
        try:
            return int(col_name.replace('Mid-', ''))
        except ValueError:
            return col_name # Return original if conversion fails
    return col_name

pop_filtered_df.columns = [clean_pop_df_column_name(col) for col in pop_filtered_df.columns]
pop_filtered_df

In [ ]:
#Output pandas in population
pop_filtered_df = pop_df.iloc[201:234]
def clean_pop_df_column_name(col_name):
    if isinstance(col_name, str) and col_name.startswith('Mid-'):
        try:
            return int(col_name.replace('Mid-', ''))
        except ValueError:
            return col_name # Return original if conversion fails
    return col_name

pop_filtered_df.columns = [clean_pop_df_column_name(col) for col in pop_filtered_df.columns]

desired_columns = [
    'Code',
    'Name',
    'Geography',
    'Area (sq km)',
    'Estimated Population mid-2022',
    '2022 people per sq. km',
    'Estimated Population mid-2019',
    '2019 people per sq. km'
]

pop_selected_columns_df = pop_filtered_df[desired_columns]
display(pop_selected_columns_df)

In [ ]:
# data in earnings
df_path_earnings = "https://data.london.gov.uk/download/2z0rk/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx"

earnings_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly")


In [ ]:
import pandas as pd

raw_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly", header=None)

# Extract the first two rows to be used as header information
header_row0 = raw_df.iloc[0] # Contains years (e.g., 2002, NaN, 2003, NaN)
header_row1 = raw_df.iloc[1] # Contains sub-headers (e.g., Code, Area, Pay (£), conf %)

# Construct new column names by combining the year and sub-header
new_columns = []
current_year = None

for i in range(len(header_row0)):
    year_val = header_row0.iloc[i]
    sub_header_val = header_row1.iloc[i]

    if i < 2: # Handle the first two columns ('Code', 'Area') specifically
        new_columns.append(str(year_val).strip())
    elif pd.isna(year_val): # If year is NaN, it's a sub-header like 'conf %' under a year
        if current_year is not None:
            new_columns.append(f"{current_year} {str(sub_header_val).strip()}")
        else:
            # This case implies a NaN year_val without a preceding year, which shouldn't happen for data columns
            new_columns.append(str(sub_header_val).strip()) # Fallback for safety
    else: # Year value is present (e.g., 2002, 2003, ...)
        current_year = int(year_val)
        new_columns.append(f"{current_year} {str(sub_header_val).strip()}")

# Create the earnings_clean DataFrame by taking data from the third row onwards
# and assigning the newly constructed column names.
earnings_clean = raw_df.iloc[2:].copy()
earnings_clean.columns = new_columns
earnings_clean = earnings_clean.reset_index(drop=True)

# Remove all 'conf %' columns
columns_to_drop = [col for col in earnings_clean.columns if 'conf %' in col]
earnings_clean = earnings_clean.drop(columns=columns_to_drop)

# Identify the columns for 'Pay (£)' for years 2011 to 2024
years_to_keep = list(range(2019, 2023))
pay_columns = [f"{year} Pay (£)" for year in years_to_keep]

# Ensure 'Code' and 'Area' are always kept
final_columns = ['Code', 'Area'] + pay_columns

# Filter earnings_clean to retain only these selected columns
earnings_clean = earnings_clean[final_columns]

earnings_clean_df = earnings_clean.iloc[0:34]

print("Cleaned earnings_clean DataFrame:")
earnings_clean_df

In [ ]:
import io
import zipfile
import requests
import pandas as pd

# Make pandas show EVERYTHING
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


def load_full_pm25_summary(zip_url):
    """
    Downloads LAEI ZIP file and returns the FULL PM2.5 Summary sheet as a DataFrame
    """
    # Download ZIP
    response = requests.get(zip_url)
    response.raise_for_status()

    # Open ZIP
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Find Excel file
        excel_name = [f for f in z.namelist() if f.lower().endswith(".xlsx")][0]

        # Load Excel
        excel_bytes = io.BytesIO(z.read(excel_name))
        df = pd.read_excel(excel_bytes, sheet_name="PM2.5 Summary")

    return df



url_2019 = (
    "https://data.london.gov.uk/download/"
    "london-atmospheric-emissions-inventory--laei--2019/"
    "17d21cd1-892e-4388-9fea-b48c1b61ee3c/"
    "LAEI-2019-Emissions-Summary-including-Forecast.zip"
)

url_2022 = (
    "https://data.london.gov.uk/download/2lg5g/4ql/"
    "LAEI2022-Emissions-Summary-Excel.zip"
)



pm25_2019_full = load_full_pm25_summary(url_2019)
pm25_2022_full = load_full_pm25_summary(url_2022)


In [ ]:

result_2022 = pm25_2022_full.iloc[[6, 77], 11:]

result_2022

In [ ]:

result_2019 = pm25_2019_full.iloc[[6, 71], 11:]

result_2019

In [ ]:
# Combining "result_2022", "2022 Pay (£)" in "earnings_clean_df", "2022 people per sq. km" in "pop_selected_columns_df", and "2022" in "vehicles_df".


In [ ]:
# Renaming "Code" to "LA Code" and "2022 Pay (£)" to "Local Authority", and filtering out rows where "LA Code" is null.

# Importing necessary libraries
import pandas as pd

df_path_earnings = "https://data.london.gov.uk/download/2z0rk/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx"

raw_df = pd.read_excel(df_path_earnings, sheet_name="Total, weekly", header=None)

# Extracting the first two rows to be used as header information
header_row0 = raw_df.iloc[0] # Contains years (e.g., 2002, NaN, 2003, NaN)
header_row1 = raw_df.iloc[1] # Contains sub-headers (e.g., Code, Area, Pay (£), conf %)

# Constructing new column names by combining the year and sub-header
new_columns = []
current_year = None

for i in range(len(header_row0)):
    year_val = header_row0.iloc[i]
    sub_header_val = header_row1.iloc[i]

    if i < 2: # Handling the first two columns ('Code', 'Area') specifically
        new_columns.append(str(year_val).strip())
    elif pd.isna(year_val): # If year is NaN, it's a sub-header like 'conf %' under a year
        if current_year is not None:
            new_columns.append(f"{current_year} {str(sub_header_val).strip()}")
        else:
            # This case implies a NaN year_val without a preceding year, which shouldn't happen for data columns
            new_columns.append(str(sub_header_val).strip()) # Fallback for safety
    else: # Year value is present (e.g., 2002, 2003, ...)
        current_year = int(year_val)
        new_columns.append(f"{current_year} {str(sub_header_val).strip()}")

# Creating the "earnings_clean" dataframe by taking data from the third row onwards and assigning the newly constructed column names.
earnings_clean = raw_df.iloc[2:].copy()
earnings_clean.columns = new_columns
earnings_clean = earnings_clean.reset_index(drop=True)

# Removing all 'conf %' columns
columns_to_drop = [col for col in earnings_clean.columns if 'conf %' in col]
earnings_clean = earnings_clean.drop(columns=columns_to_drop)

# Identifying the columns for 'Pay (£)' for years 2011 to 2024
years_to_keep = list(range(2019, 2023))
pay_columns = [f"{year} Pay (£)" for year in years_to_keep]

# Ensuring that 'Code' and 'Area' are always kept
final_columns = ['Code', 'Area'] + pay_columns

# Filtering "earnings_clean" to retain only these selected columns
earnings_clean = earnings_clean[final_columns]

earnings_clean_df = earnings_clean.iloc[0:34]

# Printing the first few rows of "earnings_2022_df"
earnings_2022_df = earnings_clean_df[['Code', 'Area', '2022 Pay (£)']].copy()
earnings_2022_df.rename(columns={'Code': 'LA Code', 'Area': 'Local Authority'}, inplace=True)
earnings_2022_df.dropna(subset=['LA Code'], inplace=True)

print("Prepared 2022 earnings data:")
earnings_2022_df.head()

In [ ]:
# Preparing the population density data for 2022. - Renaming "Code" to "LA Code" and "Name" to "Local Authority" and removing any rows with missing "LA Code"
population_2022_df = pop_selected_columns_df[['Code', 'Name', '2022 people per sq. km']].copy()
population_2022_df.rename(columns={'Code': 'LA Code', 'Name': 'Local Authority'}, inplace=True)
population_2022_df.dropna(subset=['LA Code'], inplace=True)

print("Prepared 2022 population data:")
population_2022_df.head()

In [ ]:
# Preparing traffic intensity data for 2022
vehicles_2022_df = vehicles_df[['LA Code', 'Local Authority', 2022]].copy()
vehicles_2022_df.dropna(subset=['LA Code'], inplace=True)

print("Prepared 2022 vehicles data:")
vehicles_2022_df.head()

In [ ]:
# Preparing PM2.5 air pollution data and making sure that only numeric values are present.
import pandas as pd

# Extracting borough names (excluding 'Row Labels')
borough_names = result_2022.iloc[0, 1:].values
# Extracting PM2.5 values (excluding 'Grand Total')
pm25_values = result_2022.iloc[1, 1:].values

pm25_2022_df = pd.DataFrame({
    'Local Authority': borough_names,
    '2022 PM2.5': pm25_values
})

# Filtering out specific local authority names that are not actual boroughs or are redundant
pm25_2022_df = pm25_2022_df[~pm25_2022_df['Local Authority'].isin(['Non GLA', 'Grand Total', 'City', 'City of Westminster'])].copy()

# Converting PM2.5 to numeric values
pm25_2022_df['2022 PM2.5'] = pd.to_numeric(pm25_2022_df['2022 PM2.5'])

# Maping out Local Authority names to match other dataframes where possible
name_mapping = {
    'Barking and Dagenham': 'Barking and Dagenham',
    'Barnet': 'Barnet',
    'Bexley': 'Bexley',
    'Brent': 'Brent',
    'Bromley': 'Bromley',
    'Camden': 'Camden',
    'Croydon': 'Croydon',
    'Ealing': 'Ealing',
    'Enfield': 'Enfield',
    'Greenwich': 'Greenwich',
    'Hackney': 'Hackney',
    'Hammersmith and Fulham': 'Hammersmith and Fulham',
    'Haringey': 'Haringey',
    'Harrow': 'Harrow',
    'Havering': 'Havering',
    'Hillingdon': 'Hillingdon',
    'Hounslow': 'Hounslow',
    'Islington': 'Islington',
    'Kensington and Chelsea': 'Kensington and Chelsea',
    'Kingston': 'Kingston upon Thames',
    'Lambeth': 'Lambeth',
    'Lewisham': 'Lewisham',
    'Merton': 'Merton',
    'Newham': 'Newham',
    'Redbridge': 'Redbridge',
    'Richmond': 'Richmond upon Thames',
    'Southwark': 'Southwark',
    'Sutton': 'Sutton',
    'Tower Hamlets': 'Tower Hamlets',
    'Waltham Forest': 'Waltham Forest',
    'Wandsworth': 'Wandsworth',
    'City of London': 'City of London'
}
pm25_2022_df['Local Authority'] = pm25_2022_df['Local Authority'].replace(name_mapping)

print("Prepared 2022 PM2.5 data:")
pm25_2022_df.head()

In [ ]:
# Merging income, population density, traffic intensity, and air pollution data into one dataframe for 2022.
merged_2022_data = pd.merge(earnings_2022_df, population_2022_df, on='Local Authority', how='outer')
merged_2022_data = pd.merge(merged_2022_data, vehicles_2022_df, on='Local Authority', how='outer')
merged_2022_data = pd.merge(merged_2022_data, pm25_2022_df, on='Local Authority', how='outer')

# Ensuring 'LA Code' from earnings is used as primary, or population if earnings is missing
# The first step is to clean the LA Code from "earnings_202_df", which might have a '00AA' format (placeholder format)and converting other LA Codes to object type to avoid dtype issues during merge.
merged_2022_data['LA Code_x'] = merged_2022_data['LA Code_x'].replace({'00AA': 'E09000001', '00AB': 'E09000002', '00AC': 'E09000003', '00AD': 'E09000004', '00AE': 'E09000005', '00AF': 'E09000006', '00AG': 'E09000007', '00AH': 'E09000008', '00AJ': 'E09000009', '00AK': 'E09000010', '00AL': 'E09000011', '00AM': 'E09000012', '00AN': 'E09000013', '00AP': 'E09000014', '00AQ': 'E09000015', '00AR': 'E09000016', '00AS': 'E09000017', '00AT': 'E09000018', '00AU': 'E09000019', '00AW': 'E09000020', '00AX': 'E09000021', '00AY': 'E09000022', '00AZ': 'E09000023', '00BA': 'E09000024', '00BB': 'E09000025', '00BC': 'E09000026', '00BD': 'E09000027', '00BE': 'E09000028', '00BF': 'E09000029', '00BG': 'E09000030', '00BH': 'E09000031', '00BJ': 'E09000032', '00BK': 'E09000033'})

merged_2022_data['LA Code'] = merged_2022_data['LA Code_x'].fillna(merged_2022_data['LA Code_y'])
merged_2022_data = merged_2022_data.drop(columns=['LA Code_x', 'LA Code_y'])

# Renaming columns for clarity
merged_2022_data.rename(columns={
    '2022 Pay (£)': '2022_Avg_Weekly_Earnings',
    '2022 people per sq. km': '2022_Population_Density',
    2022: '2022_Vehicle_Traffic'
}, inplace=True)

# Removing the 'LONDON' row from the population data as it is a region, not a borough.
merged_2022_data = merged_2022_data[merged_2022_data['Local Authority'] != 'LONDON']

# Removing rows where 'LA Code' is still NaN (non-borough entries that might have slipped through)
merged_2022_data.dropna(subset=['LA Code'], inplace=True)

print("Combined 2022 data for London boroughs:")
merged_2022_data.head()

In [ ]:
# Converting non-numeric values like "#" and "!" in income dataframe to NaN values and then replacing them with the mean income to make it suitable for analysis.
merged_2022_data['2022_Avg_Weekly_Earnings'] = pd.to_numeric(merged_2022_data['2022_Avg_Weekly_Earnings'], errors='coerce')
mean_earnings = merged_2022_data['2022_Avg_Weekly_Earnings'].mean()
merged_2022_data['2022_Avg_Weekly_Earnings'].fillna(mean_earnings, inplace=True)

print("Cleaned 2022_Avg_Weekly_Earnings column:")
print(merged_2022_data[['Local Authority', '2022_Avg_Weekly_Earnings']].head())

In [ ]:
merged_2022_data['2022_Avg_Weekly_Earnings'] = pd.to_numeric(merged_2022_data['2022_Avg_Weekly_Earnings'], errors='coerce')
mean_earnings = merged_2022_data['2022_Avg_Weekly_Earnings'].mean()
merged_2022_data['2022_Avg_Weekly_Earnings'] = merged_2022_data['2022_Avg_Weekly_Earnings'].fillna(mean_earnings)

print("Cleaned 2022_Avg_Weekly_Earnings column:")
print(merged_2022_data[['Local Authority', '2022_Avg_Weekly_Earnings']].head())

In [ ]:
# Printing the descriptive statistics for the merged 2022 data
print("Descriptive statistics for combined 2022 data:")
print(merged_2022_data.describe())

In [ ]:
# Displaying the merged dataframe
print("Final merged 2022 data:")
display(merged_2022_data)

In [ ]:
# Running an Ordinary Least Squares (OLS) regression by importing the "statsmodels" library.
import statsmodels.formula.api as smf

# Defining the OLS model using the specified formula and data
model = smf.ols('Q("2022 PM2.5") ~ Q("2022_Vehicle_Traffic") + Q("2022_Avg_Weekly_Earnings") + Q("2022_Population_Density")', data=merged_2022_data)

# Fitting the OLS model
results = model.fit()

print("OLS model fitted successfully.")

In [ ]:
# Printing the OLS Regression Results
print(results.summary())

In [ ]:
# Generating scatter plots for each independent variable against air pollution with regression lines
import matplotlib.pyplot as plt
import seaborn as sns

# Setting the aesthetic style of the plots
sns.set_style("whitegrid")

# Plot 1: 2022 PM2.5 vs. 2022_Vehicle_Traffic
plt.figure(figsize=(10, 6))
sns.regplot(x='2022_Vehicle_Traffic', y='2022 PM2.5', data=merged_2022_data, scatter_kws={'alpha':0.6}, line_kws={'color':'red'})
plt.title('2022 PM2.5 vs. 2022 Vehicle Traffic with Regression Line')
plt.xlabel('2022 Vehicle Traffic (All vehicles)')
plt.ylabel('2022 PM2.5 (tonnes/annum)')
plt.show()

# Plot 2: 2022 PM2.5 vs. 2022_Avg_Weekly_Earnings
plt.figure(figsize=(10, 6))
sns.regplot(x='2022_Avg_Weekly_Earnings', y='2022 PM2.5', data=merged_2022_data, scatter_kws={'alpha':0.6}, line_kws={'color':'red'})
plt.title('2022 PM2.5 vs. 2022 Average Weekly Earnings with Regression Line')
plt.xlabel('2022 Average Weekly Earnings (£)')
plt.ylabel('2022 PM2.5 (tonnes/annum)')
plt.show()

# Plot 3: 2022 PM2.5 vs. 2022_Population_Density
plt.figure(figsize=(10, 6))
sns.regplot(x='2022_Population_Density', y='2022 PM2.5', data=merged_2022_data, scatter_kws={'alpha':0.6}, line_kws={'color':'red'})
plt.title('2022 PM2.5 vs. 2022 Population Density with Regression Line')
plt.xlabel('2022 Population Density (people per sq. km)')
plt.ylabel('2022 PM2.5 (tonnes/annum)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Checking for multicollinearity
sns.heatmap(merged_2022_data[['2022_Vehicle_Traffic', '2022_Avg_Weekly_Earnings', '2022_Population_Density']].corr(), # plot a correlation matrix
            annot=True, # show the correlation values on the plot
            fmt=".2f", # set the format of the correlation values to be two decimal places
            cmap='coolwarm') # set the color palette to be coolwarm (blue for negative correlations, red for positive correlations)

plt.title('Correlation Matrix of Independent Variables (2022)') # add a title
plt.show()

In [ ]:
# calculating VIF
# This function is amended from: https://stackoverflow.com/a/51329496/4667568

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, list_var_not_to_remove=None, thresh=5):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param list_var_not_to_remove: the list of variables that should not be removed even though it has a high VIF. For example, dummy (or indicator) variables represent a categorical variable with three or more categories.
    :param thresh: the max VIF value before the feature is removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constatnt item to the data
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i)
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const as const should not be removed
        vif_df = vif_df.drop('const')

        # drop the variables that should not be removed
        if list_var_not_to_remove is not None:
            vif_df = vif_df.drop(list_var_not_to_remove)

        print('Max VIF:', vif_df.VIF.max())

        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

In [ ]:
ind_vars=['2022_Vehicle_Traffic', '2022_Avg_Weekly_Earnings', '2022_Population_Density']
X = merged_2022_data[ind_vars].copy()
X.dropna(inplace=True)
vif = drop_column_using_vif_(X, thresh=5)
print("The columns remaining after VIF selection are:")
print(vif.columns)